<a href="https://colab.research.google.com/github/ssgupta201301/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Sweta_Sengupta__LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

In [0]:
#imports
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np


In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-29 19:51:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-01-29 19:51:14 (287 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
column_headers= ['party','handicapped-infants','water-project','budget',
                 'physician-fee-freeze','el-salvador-aid','religious-groups',
                 'anti-satellite-ban','aid-to-contras','mx-missile',
                 'immigration','synfuels','education','right-to-sue','crime',
                  'duty-free','south-africa'  ]

In [9]:
df = pd.read_csv('house-votes-84.data', header=None, names=column_headers, na_values='?')
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [10]:
#recode votes as numeric
df = df.replace({'y':1,'n':0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [11]:
# how many from each party
df['party'].value_counts().sort_index()

democrat      267
republican    168
Name: party, dtype: int64

In [13]:
rep = df[df['party']=='republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [14]:
dem = df[df['party']=='democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
#Null hypothesis:
#x `1 ==x `2 where x`1 is the mean republican votes and x`2 is the mean democrat 
#votes
#Alternate hypothesis: x`1 is not equal to x`2 so the level of support between 
#two parties differ
# 95% of confidence level

In [15]:
# the percentage of republicans who voted yes
rep['handicapped-infants'].sum()/len(rep)
#len() is counting NaN values too!

0.18452380952380953

In [18]:
# remove NaN values from this column
col= rep['handicapped-infants']
np.isnan(col)
hi_no_nans=col[~np.isnan(col)]
hi_no_nans

0      0.0
1      0.0
7      0.0
8      0.0
10     0.0
      ... 
427    0.0
430    0.0
432    0.0
433    0.0
434    0.0
Name: handicapped-infants, Length: 165, dtype: float64

In [20]:
hi_no_nans.sum()/len(hi_no_nans)

0.18787878787878787

In [21]:
#mean support of rep
rep['handicapped-infants'].mean()

0.18787878787878787

In [22]:
# mean support of dem
dem['handicapped-infants'].mean()

0.6046511627906976

In [24]:
# compare with a t-test:
ttest_ind(rep['handicapped-infants'],dem['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [0]:
#account for Nan's
col= rep['handicapped-infants']
rep_hi_no_nans = col[~np.isnan(col)]
col= dem['handicapped-infants']
dem_hi_no_nans = col[~np.isnan(col)]

In [27]:
#sample size for the two samples
print(len(rep_hi_no_nans))
print(len(dem_hi_no_nans))


165
258


In [49]:
pvalue = 1.613440327937243e-18
f'{1.613440327937243e-18:.20f}'


'0.00000000000000000161'

Conclusion: Due to the p value of '0.00000000000000000161'.I reject the null hypothesis that there is no difference between average voting rates(level of support) for the Handicapped infant project.
**Handicapped infant project has higher support from democrats than republicans.**

In [0]:
# water project

In [28]:
#mean support of rep
rep['water-project'].mean()

0.5067567567567568

In [29]:
#mean support of dem
dem['water-project'].mean()

0.502092050209205

In [35]:
#Remove Nan values from this column
ttest_ind(rep['water-project'],dem['water-project'],nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
#account for Nan's
col= rep['water-project']
rep_wp_no_nans= col[~np.isnan(col)]
col=dem['water-project']
dem_wp_no_nans=col[~np.isnan(col)]


In [39]:
print(len(rep_wp_no_nans))
print(len(dem_wp_no_nans))

148
239


Conclusion: Due to the p value of 0.92 I fail to reject the null hypothesis that the republican and democrat support for the water project is different. **So the support is equal for water project.**

In [0]:
# budget

In [40]:
# mean support of rep
rep['budget'].mean()

0.13414634146341464

In [41]:
#mean support of dem
dem['budget'].mean()

0.8884615384615384

In [42]:
#compare with a t test and omit the NaNs
ttest_ind(rep['budget'],dem['budget'],nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
#account for Nan's
col= rep['budget']
rep_budget_no_nans = col[~np.isnan(col)]
col= dem['budget']
dem_budget_no_nans = col[~np.isnan(col)]

In [46]:
print(len(rep_budget_no_nans))
print(len(dem_budget_no_nans))

164
260


In [51]:
pvalue=2.0703402795404463e-77
f'{2.0703402795404463e-77:.20f}'

'0.00000000000000000000'

Conclusion: Due to the p value of 0.922.0703402795404463e-77 I reject the null hypothesis that there is no difference between average voting rates(level of support) for the Budget project.
**Budget project has higher support from democrats than republicans.**

In [0]:
#physician-fee-freeze

In [65]:
#mean support of republicans
rep['physician-fee-freeze'].mean()

0.9878787878787879

In [66]:
# mean support dems
dem['physician-fee-freeze'].mean()

0.05405405405405406

In [67]:
#compare with a t test and omit the NaNs
ttest_ind(rep['physician-fee-freeze'],dem['physician-fee-freeze'],nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
#account for Nan's
col= rep['physician-fee-freeze']
rep_pffreeze_no_nans = col[~np.isnan(col)]
col= dem['physician-fee-freeze']
dem_pffreeze_no_nans = col[~np.isnan(col)]

In [70]:
print(len(rep_pffreeze_no_nans))
print(len(dem_pffreeze_no_nans))

165
259


In [71]:
pvalue=pvalue=1.994262314074344e-177
f'{pvalue:.20f}'


'0.00000000000000000000'

Conclusion: Due to the p value of 1.994262314074344e-177, I reject the null hypothesis that there is no difference between average voting rates(level of support) for the physician-fee-freeze project.
**The physician-fee-freeze project has higher support from republicans than democrats.**

In [0]:
#el-salvador-aid

In [72]:
#mean support republicans
rep['el-salvador-aid'].mean()

0.9515151515151515

In [73]:
dem['el-salvador-aid'].mean()

0.21568627450980393

In [75]:
#compare with a t test and omit the NaNs
ttest_ind(rep['el-salvador-aid'],dem['el-salvador-aid'],nan_policy='omit')

Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

In [0]:
#account for Nan's
col= rep['el-salvador-aid']
rep_elsal_no_nans = col[~np.isnan(col)]
col= dem['el-salvador-aid']
dem_elsal_no_nans = col[~np.isnan(col)]

In [78]:
print(len(rep_elsal_no_nans))
print(len(dem_elsal_no_nans))

165
255


Conclusion: Due to the p value of 5.600520111729011e-68, I reject the null hypothesis that there is no difference between average voting rates(level of support) for the El-Salvador project. The El-Salvador project has higher support from republicans than democrats.

 ** Stretch Goals**

In [83]:
#ttest_ind(sample1,sample2,nan_policy='omit')
# writing a function to calculate ttest on each column
for col in rep.columns:
  #print(col)
  if col != 'party':
    print(str(ttest_ind(rep[col],dem[col],nan_policy='omit')) + '  result for '+ col)


Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)  result for handicapped-infants
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)  result for water-project
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)  result for budget
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)  result for physician-fee-freeze
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)  result for el-salvador-aid
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)  result for religious-groups
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)  result for anti-satellite-ban
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)  result for aid-to-contras
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)  result for mx-missile
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425

In [84]:
#trying it using dem columns
for col in dem.columns:
  if col!= 'party':
     print(str(ttest_ind(rep[col],dem[col],nan_policy='omit')) + '  result for '+ col)


Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)  result for handicapped-infants
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)  result for water-project
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)  result for budget
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)  result for physician-fee-freeze
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)  result for el-salvador-aid
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)  result for religious-groups
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)  result for anti-satellite-ban
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)  result for aid-to-contras
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)  result for mx-missile
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425

In [0]:
# Practicing 1-sample T-test
# Using only one sample, and null hyp will be different
# Looking at Democrat support of the South African bill

1a) Null Hypothesis: $\bar{x}_{1}$(average dem support for SA bill)==1
This says that 100% of democrats support this bill. Full support.

1b) Null Hypothesis: $\bar{x}_{1}$(average dem support for SA bill)==.5
This says that 59% of democrats support this bill.So the party is split.

*With 1- sample t-test I can frame that I am asking through my choice of null hypthesis*
Alterative Hypothesis:Thing you want to confirm or deny